# Train and export TensorFlow Serving model

In [1]:
def wrapper():

    import os
    import sys
    import uuid
    import random
    import tensorflow as tf
    
    from tensorflow.examples.tutorials.mnist import input_data as mnist_input_data

    from hops import model, hdfs

    training_iteration=random.randint(10, 200)
    
    mnist_dataset_folder = hdfs.copy_to_local("TourData/mnist/MNIST_data")

    # Train model
    print('Training model...')
    mnist = mnist_input_data.read_data_sets(mnist_dataset_folder, one_hot=True)
    sess = tf.InteractiveSession()
    serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
    feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
    tf_example = tf.parse_example(serialized_tf_example, feature_configs)
    x = tf.identity(tf_example['x'], name='x')  # use tf.identity() to assign name
    y_ = tf.placeholder('float', shape=[None, 10])
    w = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    sess.run(tf.global_variables_initializer())
    y = tf.nn.softmax(tf.matmul(x, w) + b, name='y')
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    values, indices = tf.nn.top_k(y, 10)
    table = tf.contrib.lookup.index_to_string_table_from_tensor(
        tf.constant([str(i) for i in range(10)]))
    prediction_classes = table.lookup(tf.to_int64(indices))
    for _ in range(training_iteration):
        batch = mnist.train.next_batch(16)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float')) 
    acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print('Training Accuracy: {}'.format(acc))
    print('Done training!')

    # Export model
    # WARNING(break-tutorial-inline-code): The following code snippet is
    # in-lined in tutorials, please update tutorial documents accordingly
    # whenever code changes.

    export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
    print('Exporting trained model to: {}'.format(export_path))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    # Build the signature_def_map.
    classification_inputs = tf.saved_model.utils.build_tensor_info(
        serialized_tf_example)
    classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
        prediction_classes)
    classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

    classification_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
              inputs={
                  tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                      classification_inputs
              },
              outputs={
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                      classification_outputs_classes,
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                      classification_outputs_scores
              },
              method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

    tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
    tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

    prediction_signature = (
          tf.saved_model.signature_def_utils.build_signature_def(
              inputs={'images': tensor_info_x},
              outputs={'scores': tensor_info_y},
              method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    builder.add_meta_graph_and_variables(
          sess, [tf.saved_model.tag_constants.SERVING],
          signature_def_map={
              'predict_images':
                  prediction_signature,
              tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                  classification_signature,
          },
          legacy_init_op=legacy_init_op)

    builder.save()

    print('Done exporting!')
    
    metrics = metrics={'accuracy': acc}

    model.export(export_path, "mnist", metrics=metrics)
    
    return metrics

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
2,application_1576832569617_0003,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
from hops import experiment

experiment.launch(wrapper, name='mnist serving example', metric_key='accuracy')

Finished Experiment 

(u'hdfs://10.0.2.15:8020/Projects/demo_deep_learning_admin000/Experiments/application_1576832569617_0003_1', {u'log': u'Experiments/application_1576832569617_0003_1/output.log', u'accuracy': 0.8574000000953674})

# Check Model Repository for best model based on accuracy

![Image7-Monitor.png](../../images/models.gif)

# Query Model Repository for best mnist Model

In [3]:
from hops import model
from hops.model import Metric
MODEL_NAME="mnist"
EVALUATION_METRIC="accuracy"

In [4]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [5]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: mnist
Model version: 1
{u'accuracy': u'0.8679999709129333'}

# Create Model Serving of Exported Model

In [6]:
from hops import serving

In [7]:
# Create serving
model_path="/Models/" + best_model['name']
response = serving.create_or_update(model_path, MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model mnist ...
Serving for model mnist successfully created

In [8]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

mnist

In [9]:
# Get serving status
serving.get_status(MODEL_NAME)

u'Running'

# Check Model Serving for active servings

![Image7-Monitor.png](../../images/servings.gif)

# Start Model Serving Server

In [10]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

In [11]:
import time
time.sleep(10) # Let the serving startup correctly

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [12]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)
NUM_FEATURES=784

In [13]:
for i in range(20):
    data = {
                "signature_name": 'predict_images',
                "instances": [np.random.rand(784).tolist()]
            }
    response = serving.make_inference_request(MODEL_NAME, data)
    print(response)

{u'predictions': [[0.375900418, 0.000395308773, 0.31800586, 0.0577337034, 0.00440815929, 0.122328237, 0.0286246073, 0.0171078555, 0.0715598464, 0.00393597]]}
{u'predictions': [[0.323040754, 0.0013824529, 0.343027502, 0.0775587931, 0.00395506946, 0.102932565, 0.0433421396, 0.0460255593, 0.054759182, 0.00397597346]]}
{u'predictions': [[0.140073, 0.0031803064, 0.51011014, 0.0738663, 0.0211428907, 0.0735798329, 0.0670034885, 0.0231796093, 0.0854813829, 0.00238307798]]}
{u'predictions': [[0.221884489, 0.000309704541, 0.299568832, 0.0390440337, 0.0265415367, 0.167707503, 0.130517393, 0.0388598628, 0.0680660158, 0.00750058]]}
{u'predictions': [[0.375826508, 0.000332933618, 0.255553842, 0.0397233441, 0.00347682112, 0.215340689, 0.0333092585, 0.024360368, 0.0507186316, 0.00135748845]]}
{u'predictions': [[0.189459398, 0.0012848539, 0.456711531, 0.0444294624, 0.0193623714, 0.110360228, 0.0615851246, 0.0247102603, 0.0868145823, 0.00528222835]]}
{u'predictions': [[0.227080435, 0.000702997379, 0.297

# Monitor inference requests from Kafka

In [14]:
from hops import kafka
from confluent_kafka import Producer, Consumer, KafkaError
config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [TOPIC_NAME]
consumer.subscribe(topics)

In [15]:
json_schema = kafka.get_schema(TOPIC_NAME)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

In [16]:
import json
for i in range(0, 10):
    msg = consumer.poll(timeout=1.0)
    if msg is not None:
        value = msg.value()
        try:
            event_dict = kafka.parse_avro_msg(value, avro_schema)
            prediction = json.loads(event_dict["inferenceResponse"])["predictions"][0]
            print("serving: {}, version: {}, timestamp: {},"\
                  "\nrequest: {},\nprediction:{}, http_response_code: {},"\
                  " serving_type: {}\n".format(
                       event_dict["modelName"],
                       event_dict["modelVersion"],
                       event_dict["requestTimestamp"],
                       event_dict["inferenceRequest"],
                       prediction,
                       event_dict["responseHttpCode"],
                       event_dict["servingType"]
            ))
        except Exception as e:
            print("A message was read but there was an error parsing it")
            print(e)
    else:
        print("timeout.. no more messages to read from topic")

timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
serving: mnist, version: 1, timestamp: 1576839216010,
request: {"instances": [[0.26221002712252517, 0.49626041876817373, 0.42672360842175927, 0.8814280155056613, 0.5157214764316816, 0.8351948879334856, 0.327921993703124, 0.6175404947931116, 0.15918178639985558, 0.12523810217015974, 0.07250759498156811, 0.41936138833963854, 0.2961269006220806, 0.012320537325177794, 0.3738800175005268, 0.5096934532848266, 0.9958337133322065, 0.21405235999876382, 0.20394434096492908, 0.6487246069828851, 0.9441438024612918, 0.4725274818997742, 0.6945817459576431, 0.8014472838658498, 0.8147819842900006, 0.4914487885525748, 0.01653979795406435, 0.08576942298748813, 0.5387473491767208, 0.31510